In [1]:
import pandas as pd
import json
import requests
import datetime
import locale

In [2]:
MAPPING_CODES_TYPES_PRET = {'NOR': 'Prêt normal',
                            'COU': 'Prêt court',
                            'SP': 'Consultable sur place',
                            'RDV': 'Consultable sur place sur rendez-vous',
                            'NC': 'Non consultable',
                            'PCCOU': 'PC prêt 1j',
                            'PCLONG': 'PC prêt long',
                            'PCPRO': 'PC prêt professionnel',
                            'EL': 'En ligne'}
MAPPING_BIBS = {"LASH": "BU Lettres Arts Sciences Humaines",
                "DROIT": "BU Droit et Science politique",
                "SJA": "BU Saint-Jean d’Angély",
                "SCIEN": "BU Sciences",
                "MEDP": "BU Médecine",
                "STAPS": "BU STAPS",
                "LCSOPHIA": "Learning Center SophiaTech",
                "BIUTF": "Bibliothèque de l’IUT Fabron",
                "BIUTM": "Bibliothèque de l’IUT de Menton",
                "BIUTC": "Bibliothèque de l’IUT de Cannes",
                "VA": "Bibliothèque de la Villa Arson",
                "OCANI": "Bibliothèque de l’OCA Mont Gros",
                "OCAVV": "Bibliothèque de l’OCA Lagrange",
                "OCASA": "Bibliothèque de l’OCA Géoazur",
                "JAD": "Bibliothèque du LJAD",
                "LIRCE": "Bibliothèque du LIRCES",
                "CMMC": "Bibliothèque du CMMC",
                "BCL": "Bibliothèque du BCL"}
MAPPING_LOCS = {"A213E": "Salle A213 - Enseignement-Vulgarisation",
"A213G": "Salle A213 - Guides-Dico",
"A213I": "Salle A213 - Informatique",
"A213O": "Salle A213 - Ouvrages",
"A213P": "Salle A213 - Revues",
"A214": "Salle A214",
"A216C": "Salle A216 - Cartes-Atlas",
"A216P": "Salle A216 - Revues",
"A216T": "Salle A216 - Thèses-Mémoires",
"AGM": "1er étage - Agrégation Maths",
"ARC": "Archives",
"ASE": "RDC - Fonds ASEMI",
"ATE": "Ateliers Villa Arson",
"BCL": "Bât. Horloge",
"BDOC": "Bureau documentation",
"BDP": "Accueil",
"BDVIDEO": "DVD, Vidéos, BD",
"BOSCO": "Fonds Henri Bosco - Bibliothèque personnelle",
"BP0": "RDC - Bureau de prêt",
"BUR": "Bureau",
"BUR2": "Bureau de prêt",
"BUR3": "Bureau",
"BUR4": "Bureau",
"CA": "Casque audio (à l'accueil)",
"CAL": "Calern",
"CAR": "RDC - Cartes",
"CDR1": "RDC - Accueil",
"CDS": "CD-ROM",
"CHATT": "Fonds Chatterji",
"CLT": "RDC - Romans et BD",
"CM": "PC portables (à l'accueil)",
"CMMC": "CMMC",
"CO": "Compactus",
"CODE": "Etage (SA2) - Codes",
"DET": "Détente",
"DET0": "RDC - Détente",
"DET1": "1er étage - Espace détente",
"DOC": "1er étage - Documentation professionnelle",
"DOCEL": "RDC - Bureau de prêt",
"DRO": "1er étage",
"ECO": "1er étage",
"EDN": "EDN",
"ENS": "Salle de lecture",
"EPS7": "Dossiers EPS",
"ES": "Espace Sécurisé (PC portables)",
"ETHIQ": "Ethique en santé",
"FLE": "1er étage - Fonds FLE",
"FP1": "Fonds patrimoniaux (1er étage)",
"FP2": "Fonds patrimoniaux (magasin)",
"FPAT": "Magasins - Fonds patrimoniaux",
"FS": "Fonds spécial",
"GEST": "1er étage",
"IDEM": "Magasins",
"IFSI": "Sciences infirmières",
"KINE": "RDC - Kinésithérapie",
"LANG7": "Espace Langues",
"LAS": "LAS",
"LELEC": "Livres électroniques en ligne",
"LIRCE": "LIRCES",
"LLA": "Magasins-Fonds LLAOR",
"MAG": "Magasins",
"MAG2": "Magasins (sous-sol)",
"MAN": "RDC - Manuels",
"MED": "RDC",
"MELEC": "Mémoires électroniques",
"MEM": "Mémoires (bacs)",
"MEZA": "Mezzanine-Atlas",
"MEZO": "Mezzanine - Ouvrages anciens",
"MLC": "Magasins",
"MM7": "Mémoires (Salle de lecture)",
"MSH": "2e étage - bibliothèque de recherche",
"ODONT": "RDC - Odontologie",
"ODYS": "RDC - Fonds Société et environnement",
"ORTHO": "Orthophonie",
"PASS0": "Villa Passiflores (0)",
"PASS1": "Villa Passiflores (1)",
"PDCTG": "RDC (SA1) - Presse",
"PDMAG": "Magasins",
"PDSA0": "Sous-sol (SA0) - Revues",
"PDSA1": "RDC (SA1) - Revues",
"PDSA2": "Etage (SA2) - Revues",
"PDSDR": "Sous-sol - Revues - Salle de recherche",
"PELEC": "Périodiques électroniques",
"PER7": "Revues",
"PERS": "Revues",
"PHALL": "Hall - Périodiques",
"PJEC2": "2e étage - revues d'Economie et Gestion",
"PJECM": "Magasins",
"PJECO": "1er étage",
"PJODM": "Magasins",
"PJODO": "RDC - Odontologie",
"PJPRS": "RDC - Presse",
"PJSOC": "2e étage - revues de Sociologie",
"PKINE": "RDC - Kinésithérapie",
"PLASE": "Magasins- Fonds ASEMI - Revues",
"PLIDE": "2e étage - revues Fonds IDERIC",
"PLLLA": "Magasins - Fonds LLAOR - Revues",
"PLMG1": "Magasins - Revues",
"PLMG2": "Magasins - Compactus",
"PMMAG": "Magasins",
"PMPAS": "Revues",
"PREF0": "Hall - Kiosque",
"PSBUR": "RDC Bureau Catalogage - Revues",
"PSMG2": "Magasins - Revues",
"PSNI2": "1er étage - Revues",
"PSPRS": "RDC - Presse",
"RDC": "Rez de chaussée",
"RE1": "RDC - Dictionnaires",
"RE2": "1er étage - Dictionnaires",
"REF0": "RDC - Dictionnaires",
"REF1": "1er étage - Dictionnaires",
"RF7": "Espace Langues",
"RFS": "Dictionnaires",
"RUS": "Rustrel",
"SA0": "Sous-sol",
"SA1": "RDC",
"SA2": "Etage",
"SCPO": "1er étage",
"SDR": "Sous-sol - Salle de recherche",
"SHL": "RDC",
"SL": "Salle de lecture",
"SL1": "RDC",
"SL2": "1er étage",
"SL7": "Salle de lecture",
"SLC": "Salle de lecture - Collections",
"SLO": "Salle de lecture - Ouvrages",
"SLP": "Salle de lecture - Revues",
"SLT": "Salle de lecture - Thèses-Mémoires",
"SLU": "Salle de lecture - Usuels-Doc. OCA",
"SS1O": "Sous-sol 1 - Ouvrages",
"SS1P": "Sous-sol 1 - Revues",
"SS2": "Sous-sol 2",
"ST": "RDC",
"TELEC": "Thèses électroniques",
"TH7": "Thèses (Salle de lecture)",
"THECO": "Magasins",
"THLAS": "Magasins",
"THODO": "Magasins",
"URF": "Urfist",
"US": "Usuels",
"US0": "Sous-sol (SA0) - Dictionnaires",
"US1": "RDC (SA1) - Dictionnaires",
"US2": "Etage (SA2) - Dictionnaires",
}
mapping_codes_types_pret = MAPPING_CODES_TYPES_PRET
mapping_bibs = MAPPING_BIBS
mapping_locs = MAPPING_LOCS

In [6]:
url = "https://catalogue-bu-cotedazur-koha.preprod.biblibre.eu/api/v1/public/biblios/105813/items"
response = requests.request("GET", url).text
data = json.loads(response)
sorted(data, key=lambda x: BIBS_ORDER[x.get('home_library_id')])
#l = data.sort(key=lambda x: BIBS_ORDER[x["home_library_id"]])
#l

#sorted(data, key=lambda x: BIBS_ORDER.get(x[0].get('home_library_id')))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
# Set the locale to French
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

def convert_to_french_date(date_string):
    # Convert the date string to a datetime object
    date = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    # Format the date as a string in French
    date_string_french = date.strftime('%d %B %Y')
    return date_string_french

def extract_koha_item(item):
    result = {}
    # s'il n'y a pas de date de retour et si le doc n'est pas en transfert -> disponible
    if (item['checked_out_date'] is None) & (item["holding_library_id"] == item["home_library_id"]):
        result['availability'] = 'Disponible'
    # s'il n'y a pas de date de retour et si le doc est en transfert -> indisponible
    elif (item['checked_out_date'] is None) & (item["holding_library_id"] != item["home_library_id"]):
        result['availability'] = 'Indisponible : en transit'
    # sinon (ie s'il y a une date de retour) -> indisponible
    #elif item['checked_out_date'] is not None:
    else:
        checked_out_date = convert_to_french_date(item['checked_out_date'])
        result['availability'] = f'Indisponible : emprunté (Retour le {checked_out_date})'
    result["home_library_id"] = mapping_bibs[item["home_library_id"]]
    result["location"] = mapping_locs[item["location"]]
    result["callnumber"] = item["callnumber"]
    result["loan_type"] = mapping_codes_types_pret[item["item_type_id"]]
    # si pério on affiche l' état de coll et la règle deprêt
    if item["external_id"].startswith('HDL'):
        result["serial_issue_number"] = f"Etat de collection : {item['serial_issue_number']}"
    elif item["serial_issue_number"] is not None:
        result["serial_issue_number"] =  item["serial_issue_number"]
    return result

In [7]:
url = "https://catalogue-bu-univ-cotedazur.biblibre.fr/api/v1/public/biblios/105813/items"
response = requests.request("GET", url).text
data = json.loads(response)
new_data = [extract_koha_item(i) for i in data] 
new_data

[{'availability': 'Disponible',
  'home_library_id': 'BU Sciences',
  'location': 'RDC - Manuels',
  'callnumber': '577.85 SEL',
  'loan_type': 'Prêt normal'},
 {'availability': 'Disponible',
  'home_library_id': 'BU Sciences',
  'location': 'RDC - Manuels',
  'callnumber': '577.85 SEL',
  'loan_type': 'Prêt normal'},
 {'availability': 'Disponible',
  'home_library_id': 'BU Sciences',
  'location': 'RDC - Manuels',
  'callnumber': '577.85 SEL',
  'loan_type': 'Prêt normal'},
 {'availability': 'Disponible',
  'home_library_id': 'BU Sciences',
  'location': 'RDC - Manuels',
  'callnumber': '577.85 SEL',
  'loan_type': 'Prêt normal'},
 {'availability': 'Disponible',
  'home_library_id': 'BU Sciences',
  'location': 'RDC - Manuels',
  'callnumber': '577.85 SEL',
  'loan_type': 'Prêt normal'},
 {'availability': 'Disponible',
  'home_library_id': 'BU Sciences',
  'location': 'RDC - Manuels',
  'callnumber': '577.85 SEL',
  'loan_type': 'Prêt normal'},
 {'availability': 'Indisponible : emprun

In [14]:
for dic in data:
    for val,cal in dic.items():
        print(val,cal)

acquisition_date 2019-07-11
biblio_id 273173
callnumber 343.072 ROD
checked_out_date None
copy_number None
damaged_status 0
external_id 0962099516
holding_library_id DROIT
home_library_id DROIT
item_id 500130
item_type_id NOR
location PASS0
lost_status 0
not_for_loan_status 0
public_notes None
restricted_status None
serial_issue_number None
uri None
withdrawn 0
acquisition_date 2019-07-11
biblio_id 273173
callnumber 343.072 ROD
checked_out_date None
copy_number None
damaged_status 0
external_id 0962099517
holding_library_id DROIT
home_library_id DROIT
item_id 500131
item_type_id NOR
location PASS0
lost_status 0
not_for_loan_status 0
public_notes None
restricted_status None
serial_issue_number None
uri None
withdrawn 0
acquisition_date 2019-07-11
biblio_id 273173
callnumber 343.072 ROD
checked_out_date None
copy_number None
damaged_status 0
external_id 0962099518
holding_library_id LASH
home_library_id DROIT
item_id 500132
item_type_id NOR
location PASS1
lost_status 0
not_for_loan_statu